In [1]:
import sys, itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import random

In [2]:
num_dims = 1; convert_data = 0; 
percentile = 50; threshold = 0; labelcol = -1; split_ratio = 0.9

In [29]:
data = pd.read_csv("Data/Iris.csv", index_col=0)

In [30]:
labelcol = 'class label'
grouped = data.groupby(data.loc[:,labelcol])

In [40]:
def drop_col(data, col):
    return data.drop(col, axis = 1)

In [41]:
traindata = []
testdata = []
grouped = data.groupby(data.loc[:,labelcol])
classes = [c for c in grouped.groups.keys()]
classwise = {} 
for c in classes:
    classwise[c] = grouped.get_group(c)
    rows = random.sample(list(classwise[c].index), 
                                int(classwise[c].shape[0] * 
                                split_ratio))
    traindata.append(classwise[c].loc[rows])
    testdata.append(classwise[c].drop(rows))

traindata = pd.concat(traindata)
testdata = pd.concat(testdata)

In [42]:
means = {}
classes = [c for c in grouped.groups.keys()]
for c in classes:
    means[c] = np.array(drop_col(classwise[c], labelcol).mean(axis = 0))
# calculate the overall mean of all the data
overall_mean = np.array(drop_col(data, labelcol).mean(axis = 0))


In [70]:
# calculate between class covariance matrix
# S_B = \sigma{N_i (m_i - m) (m_i - m).T}
S_B = np.zeros((data.shape[1] - 1, data.shape[1] - 1))
for c in means.keys():
    S_B += np.multiply(len(classwise[c]),
                        np.outer((means[c] - overall_mean), 
                                (means[c] - overall_mean)))


In [91]:

# calculate within class covariance matrix
# S_W = \sigma{S_i}
# S_i = \sigma{(x - m_i) (x - m_i).T}
S_W = np.zeros(S_B.shape) 
for c in classes: 
    tmp = np.subtract(drop_col(classwise[c], labelcol).T, np.expand_dims(means[c], axis=1))
    S_W = np.add(np.dot(tmp, tmp.T), S_W)

# objective : find eigenvalue, eigenvector pairs for inv(S_W).S_B
mat = np.dot(np.linalg.pinv(S_W), S_B)
eigvals, eigvecs = np.linalg.eig(mat)
eiglist = [(eigvals[i], eigvecs[:, i]) for i in range(len(eigvals))]

# sort the eigvals in decreasing order
eiglist = sorted(eiglist, key = lambda x : x[0], reverse = True)

# take the first num_dims eigvectors
w = np.array([eiglist[i][1] for i in range(num_dims)])

w = w
means = means

In [97]:
list(means.keys())

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

In [99]:
tot = 0
for c in means.keys():
    tot += np.dot(w, means[c])
w0 = 0.5 * tot
# for 2 classes case; mark if class 1 is >= w0 or < w0
c1 = list(means.keys())[0]
c2 = list(means.keys())[1]
mu1 = np.dot(w, means[c1])
if (mu1 >= w0):
    c1 = 'ge'
else:
    c1 = 'l'

In [100]:
c1

'l'